In [11]:
import os
import pandas as pd 
import cv2  
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import shutil

In [53]:
#train_path = '../data/samples10/org'
train_path = '../data/train'
cameras = os.listdir(train_path)

train_images = []
for camera in cameras:
    for fname in sorted(os.listdir(train_path+ '/'+ camera)):
        train_images.append((camera, fname))

train = pd.DataFrame(train_images, columns=['camera', 'fname'])
print(train.shape)
train.sample(5)

(2750, 2)


,camera,fname
248,HTC-1-M7,(HTC-1-M7)75.jpg
744,iPhone-6,(iP6)274.jpg
1645,Motorola-Nexus-6,(MotoNex6)95.jpg
1093,LG-Nexus-5x,(LG5x)93.jpg
2038,Samsung-Galaxy-Note3,(GalaxyN3)200.jpg


In [54]:
#base = '../data/samples10'
base = '../data/train'
input_path = base 
output_path = base+'-m'

def read(input_path,camera,fname):
    image_path = input_path+"/"+camera+'/'+fname
    #print(image_path)
    image = cv2.imread(image_path)
    return image

def write(image,output_path,camera,fname,manip):
    image_path = output_path +'/'+ camera +'/'+ os.path.splitext(fname)[0]
    if(manip == 'crop' ):
        image_path = image_path+ '-512.jpg'
        print(image_path)
        cv2.imwrite(image_path, image)

def gamma_correction(img, correction):
    img = img/255.0
    img = cv2.pow(img, correction)
    return np.uint8(img*255)

def crop(image,output_path,camera,fname):
    h, w, c = image.shape
    x1 = (w//2 - 256)
    y1 = (w//2 + 256)
    x2 = (h//2 - 256)
    y2 = (h//2 + 256)
    cropped_image = image[x2:y2, x1:y1]
    #print(cropped_image.shape)
    write(cropped_image,output_path,camera,fname,'crop')

def comp(image,output_path,camera,fname):
    for j in (70,90):
        fullname = output_path +'/'+ camera +'/'+ os.path.splitext(fname)[0] +'-j'+str(j)+ '.jpg'
        flname = os.path.splitext(fname)[0] +'-j'+str(j)+ '.jpg'
        #print(fullname)
        cv2.imwrite(fullname, image, [int(cv2.IMWRITE_JPEG_QUALITY), j])
        image = read(output_path,camera,flname)
        crop(image,output_path,camera,flname)
        os.remove(fullname)
        
def resize(image,output_path,camera,fname):
    for r in (0.5,0.8,1.0,2.0):
        flname = os.path.splitext(fname)[0] +'-r'+ str(r)+ '.jpg'
        #print(flname)
        resize = cv2.resize(image,None,fx=r, fy=r, interpolation = cv2.INTER_CUBIC)
        crop(resize,output_path,camera,flname)

def gamma(image,output_path,camera,fname):
    for g in (0.8,1.2):
        flname = os.path.splitext(fname)[0] +'-g'+ str(g)+ '.jpg'
        gc = gamma_correction(image, g)
        crop(gc,output_path,camera,flname)

def cleanupcreatedir(imagefolders):
    print(output_path)
    if os.path.exists(output_path):
        shutil.rmtree(output_path)
        print('Cleaned :'+output_path)
    if not os.path.exists(output_path):
        os.makedirs(output_path)
        print('Created :'+output_path)
    for imagefolder in np.unique(imagefolders):
        folder = output_path +'/'+imagefolder
        if not os.path.exists(folder):
            os.makedirs(folder)
            print('Created :'+folder)
            
def preprocess():
    cleanupcreatedir(train['camera'])
    for camera,fname in zip(train['camera'],train['fname']):
        image = read(input_path,camera,fname)
        crop(image,output_path,camera,fname)
        comp(image,output_path,camera,fname)
        resize(image,output_path,camera,fname)
        gamma(image,output_path,camera,fname)
        
        
preprocess()

../data/train-m
Created :../data/train-m
Created :../data/train-m/HTC-1-M7
Created :../data/train-m/LG-Nexus-5x
Created :../data/train-m/Motorola-Droid-Maxx
Created :../data/train-m/Motorola-Nexus-6
Created :../data/train-m/Motorola-X
Created :../data/train-m/Samsung-Galaxy-Note3
Created :../data/train-m/Samsung-Galaxy-S4
Created :../data/train-m/Sony-NEX-7
Created :../data/train-m/iPhone-4s
Created :../data/train-m/iPhone-6
../data/train-m/HTC-1-M7/(HTC-1-M7)1-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)1-j70-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)1-j90-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)1-r0.5-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)1-r0.8-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)1-r1.0-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)1-r2.0-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)1-g0.8-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)1-g1.2-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)10-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)10-j70-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-

../data/train-m/HTC-1-M7/(HTC-1-M7)114-j70-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)114-j90-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)114-r0.5-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)114-r0.8-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)114-r1.0-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)114-r2.0-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)114-g0.8-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)114-g1.2-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)115-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)115-j70-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)115-j90-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)115-r0.5-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)115-r0.8-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)115-r1.0-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)115-r2.0-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)115-g0.8-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)115-g1.2-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)116-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)116-j70-512.jpg
../data/train-m/HTC-1-M7/(H

../data/train-m/HTC-1-M7/(HTC-1-M7)130-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)130-j70-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)130-j90-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)130-r0.5-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)130-r0.8-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)130-r1.0-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)130-r2.0-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)130-g0.8-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)130-g1.2-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)131-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)131-j70-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)131-j90-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)131-r0.5-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)131-r0.8-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)131-r1.0-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)131-r2.0-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)131-g0.8-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)131-g1.2-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)132-512.jpg
../data/train-m/HTC-1-M7/(HTC-1

../data/train-m/HTC-1-M7/(HTC-1-M7)146-g1.2-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)147-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)147-j70-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)147-j90-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)147-r0.5-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)147-r0.8-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)147-r1.0-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)147-r2.0-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)147-g0.8-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)147-g1.2-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)148-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)148-j70-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)148-j90-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)148-r0.5-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)148-r0.8-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)148-r1.0-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)148-r2.0-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)148-g0.8-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)148-g1.2-512.jpg
../data/train-m/HTC-1-M7/(

../data/train-m/HTC-1-M7/(HTC-1-M7)162-g0.8-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)162-g1.2-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)163-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)163-j70-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)163-j90-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)163-r0.5-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)163-r0.8-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)163-r1.0-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)163-r2.0-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)163-g0.8-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)163-g1.2-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)164-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)164-j70-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)164-j90-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)164-r0.5-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)164-r0.8-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)164-r1.0-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)164-r2.0-512.jpg
../data/train-m/HTC-1-M7/(HTC-1-M7)164-g0.8-512.jpg
../data/train-m/HTC-1-M7/(

...

../data/train-m/Sony-NEX-7/(Nex7)27-r2.0-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)27-g0.8-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)27-g1.2-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)270-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)270-j70-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)270-j90-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)270-r0.5-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)270-r0.8-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)270-r1.0-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)270-r2.0-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)270-g0.8-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)270-g1.2-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)271-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)271-j70-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)271-j90-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)271-r0.5-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)271-r0.8-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)271-r1.0-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)271-r2.0-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)271-g0.8-512.jpg
../data/train-m/S

../data/train-m/Sony-NEX-7/(Nex7)39-r2.0-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)39-g0.8-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)39-g1.2-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)4-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)4-j70-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)4-j90-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)4-r0.5-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)4-r0.8-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)4-r1.0-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)4-r2.0-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)4-g0.8-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)4-g1.2-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)40-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)40-j70-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)40-j90-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)40-r0.5-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)40-r0.8-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)40-r1.0-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)40-r2.0-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)40-g0.8-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)40-g1.2-51

../data/train-m/Sony-NEX-7/(Nex7)56-r2.0-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)56-g0.8-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)56-g1.2-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)57-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)57-j70-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)57-j90-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)57-r0.5-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)57-r0.8-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)57-r1.0-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)57-r2.0-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)57-g0.8-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)57-g1.2-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)58-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)58-j70-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)58-j90-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)58-r0.5-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)58-r0.8-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)58-r1.0-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)58-r2.0-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)58-g0.8-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)5

../data/train-m/Sony-NEX-7/(Nex7)73-r2.0-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)73-g0.8-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)73-g1.2-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)74-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)74-j70-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)74-j90-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)74-r0.5-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)74-r0.8-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)74-r1.0-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)74-r2.0-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)74-g0.8-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)74-g1.2-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)75-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)75-j70-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)75-j90-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)75-r0.5-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)75-r0.8-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)75-r1.0-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)75-r2.0-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)75-g0.8-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)7

../data/train-m/Sony-NEX-7/(Nex7)90-r2.0-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)90-g0.8-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)90-g1.2-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)91-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)91-j70-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)91-j90-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)91-r0.5-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)91-r0.8-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)91-r1.0-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)91-r2.0-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)91-g0.8-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)91-g1.2-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)92-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)92-j70-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)92-j90-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)92-r0.5-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)92-r0.8-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)92-r1.0-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)92-r2.0-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)92-g0.8-512.jpg
../data/train-m/Sony-NEX-7/(Nex7)9